# 02 - Test Génération LLM

Notebook pour tester et ajuster les prompts de génération de synthèses.

## Objectifs
1. Charger le ground truth (élèves + synthèses prof)
2. Tester le prompt builder
3. Générer des synthèses et comparer avec ground truth
4. Itérer sur les prompts

In [1]:
# ruff: noqa: E402
import sys
from pathlib import Path

# Auto-détecter project_root
current = Path.cwd()
while current != current.parent:
    if (current / "pyproject.toml").exists():
        project_root = current
        break
    current = current.parent

sys.path.insert(0, str(project_root))

# Configurer le logging
from src.core.logging_config import setup_batch_logging

logger, batch_id, log_file = setup_batch_logging(batch_id="generation_test")

# Paths
DATA_DIR = project_root / "data"
GROUND_TRUTH_PATH = DATA_DIR / "ground_truth" / "chiron_ground_truth.json"

print(f"Project root: {project_root}")

INFO     | Logs sauvegardés dans : c:\Users\Florent\Documents\data_science\chiron\data\processed\logs\batch_generation_test.log
INFO     | 🆔 Batch ID : generation_test
Project root: c:\Users\Florent\Documents\data_science\chiron


## 1. Charger les données

In [2]:
from src.core.data_loader import load_ground_truth

dataset = load_ground_truth(GROUND_TRUTH_PATH)
eleves = dataset.eleves

print(f"Chargé {len(eleves)} élèves")
print(f"Classe: {dataset.metadata.classe} - {dataset.metadata.trimestre}")
for e in eleves:
    print(f"  - {e.eleve_id}: {len(e.matieres)} matières")

Chargé 4 élèves
Classe: 5ème - T1
  - ELEVE_A: 12 matières
  - ELEVE_B: 11 matières
  - ELEVE_C: 12 matières
  - ELEVE_D: 11 matières


## 2. Test du Prompt Builder

In [3]:
from src.generation.prompt_builder import PromptBuilder, format_eleve_data

# Visualiser les données formatées d'un élève
eleve_test = eleves[0]
print("=" * 60)
print("DONNÉES ÉLÈVE FORMATÉES")
print("=" * 60)
print(format_eleve_data(eleve_test))

INFO     | Table llm_metrics créée/vérifiée
INFO     | MetricsCollector initialisé avec DB: c:\Users\Florent\Documents\data_science\chiron\data\processed\llm_metrics\metrics.duckdb
DONNÉES ÉLÈVE FORMATÉES
Élève : ELEVE_A
Genre : Fille
Absences : 4 demi-journées (justifiées)
Engagements : Déléguée titulaire

RÉSULTATS PAR MATIÈRE :
----------------------------------------
• Anglais LV1: 15.21/20 (classe: 10.83, +4.38)
  "Bons résultats. ELEVE_A fournit un travail régulier et sérieux à la maison tout comme en classe. L'attitude est toujours positive et constructive. Poursuivez ainsi!"
• Arts Plastiques: 15.00/20 (classe: 14.92, +0.08)
  "Bon ensemble, bilan satisfaisant, continuez ainsi en restant concentrée."
• EPS: 8.00/20 (classe: 12.79, -4.79)
  "Bilan très insuffisant, ELEVE_A n'a pas réussi à s'orienter avec efficacité. Elle a beaucoup marché et s'est dispersée avec son groupe. Son travail a manqué de sérieux et d'implication dans les défis à relever en course d'orientation. De sin

In [4]:
# Créer un prompt builder avec few-shot (2 exemples)
exemples_fewshot = eleves[:2]  # ELEVE_A et ELEVE_B comme exemples
eleve_cible = eleves[2]  # ELEVE_C à générer

builder = PromptBuilder(exemples=exemples_fewshot)

# Afficher le prompt complet (tronqué)
print("=" * 60)
print("PROMPT COMPLET (pour debug)")
print("=" * 60)
prompt_text = builder.build_prompt_text(eleve_cible, classe_info="5ème - Trimestre 1")
# Afficher seulement le début du system prompt
lines = prompt_text.split("\n")
print("\n".join(lines[:50]))  # Premières 50 lignes
print(f"\n... ({len(prompt_text)} caractères total, {len(lines)} lignes)")

PROMPT COMPLET (pour debug)
[SYSTEM]
Tu es un assistant pour professeur principal de collège/lycée.
Ta tâche est de rédiger des synthèses trimestrielles et d'identifier les points clés pour le conseil de classe.

## ÉTAPE 1 : RÉDIGER LA SYNTHÈSE

Consignes pour la synthèse :
- Adopte le même ton et style que les exemples fournis par le professeur
- Cite les matières concernées SANS mentionner les notes chiffrées
- Identifie les points forts ET les axes d'amélioration
- Sois constructif et bienveillant
- Utilise le vouvoiement ou tutoiement selon les exemples
- Accorde correctement selon le genre de l'élève (il/elle, lui/elle)
- Longueur : 2-4 phrases, concises et percutantes
- NE CITE JAMAIS les notes exactes (ex: "12,5/20")

## ÉTAPE 2 : IDENTIFIER LES ALERTES

Une alerte signale un point nécessitant une attention particulière.
Critères :
- Note < 10/20 dans une matière
- Note très inférieure à la moyenne de classe (écart > 3 points)
- Comportement problématique mentionné dans les app

In [5]:
# Afficher la synthèse ground truth pour comparaison
print("=" * 60)
print(f"SYNTHÈSE GROUND TRUTH ({eleve_cible.eleve_id})")
print("=" * 60)
print(eleve_cible.synthese_ground_truth)

SYNTHÈSE GROUND TRUTH (ELEVE_C)
Bilan globalement positif mais perfectible. ELEVE_C a fourni un travail sérieux dans de nombreuses matières. Nous comptons désormais sur lui, pour améliorer sa participation en classe et intensifier son travail personnel en SVT et en Anglais afin de réaliser une belle année de 5ème. Nous l'en savons capable.


## 3. Génération avec LLM (JSON structuré)

Le LLM retourne un JSON structuré avec :
- `synthese_texte` : Le texte de la synthèse
- `alertes` : Points d'attention (matière, description, sévérité urgent/attention)
- `reussites` : Points forts (matière, description)
- `posture_generale` : actif / passif / perturbateur / variable
- `axes_travail` : Axes prioritaires d'amélioration

⚠️ Nécessite une clé API valide dans `.env`

In [6]:
# Vérifier que les clés API sont configurées
import os

from dotenv import load_dotenv

load_dotenv(project_root / ".env")

has_openai = bool(os.getenv("OPENAI_API_KEY"))
has_anthropic = bool(os.getenv("ANTHROPIC_API_KEY"))

print(f"OpenAI API key: {'✓' if has_openai else '✗'}")
print(f"Anthropic API key: {'✓' if has_anthropic else '✗'}")

if not (has_openai or has_anthropic):
    print("\n⚠️ Aucune clé API configurée. Créez un fichier .env à la racine.")

OpenAI API key: ✓
Anthropic API key: ✗


In [7]:
# Générer une synthèse
from src.generation.generator import SyntheseGenerator

# Choisir le provider selon les clés disponibles
provider = "openai" if has_openai else "anthropic"
model = "gpt-5-mini" if provider == "openai" else "claude-3-5-haiku-latest"

print(f"Utilisation de {provider}/{model}")

generator = SyntheseGenerator(provider=provider, model=model)
generator.set_exemples(exemples_fewshot)

Utilisation de openai/gpt-5-mini
INFO     | SimpleRateLimiter initialisé: 500 RPM (verbose=True)
INFO     | ✨ Rate limiter PARTAGÉ créé pour openai: 500 RPM (singleton)
INFO     | SimpleRateLimiter initialisé: 50 RPM (verbose=True)
INFO     | ✨ Rate limiter PARTAGÉ créé pour anthropic: 50 RPM (singleton)
INFO     | SimpleRateLimiter initialisé: 100 RPM (verbose=True)
INFO     | ✨ Rate limiter PARTAGÉ créé pour mistral: 100 RPM (singleton)
INFO     | LLMManager initialisé avec rate limiters partagés
INFO     | Few-shot configuré avec 2 exemple(s)


In [8]:
# Générer pour ELEVE_C
logger.info(f"Génération synthèse pour {eleve_cible.eleve_id}")

synthese = generator.generate(
    eleve_cible,
    classe_info="5ème - Trimestre 1",
)

print("=" * 60)
print("SYNTHÈSE GÉNÉRÉE")
print("=" * 60)
print(f"\n📝 Texte:\n{synthese.synthese_texte}")
print(f"\n🚨 Alertes ({len(synthese.alertes)}):")
for a in synthese.alertes:
    print(f"  - [{a.severite.upper()}] {a.matiere}: {a.description}")
print(f"\n✅ Réussites ({len(synthese.reussites)}):")
for r in synthese.reussites:
    print(f"  - {r.matiere}: {r.description}")
print(f"\n🎭 Posture générale: {synthese.posture_generale}")
print(f"\n🎯 Axes de travail: {', '.join(synthese.axes_travail)}")

INFO     | Génération synthèse pour ELEVE_C
INFO     | Génération synthèse pour ELEVE_C via openai/gpt-5-mini
INFO     | OpenAIClient initialisé avec modèle: gpt-5-mini
INFO     | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     | OpenAI call réussi: gpt-5-mini - 4717 tokens - 29599ms - Content length: 1888 chars
INFO     | Synthèse générée: 491 chars, 2 alertes, 6 réussites, posture=passif, 4717 tokens
SYNTHÈSE GÉNÉRÉE

📝 Texte:
ELEVE_C montre de belles réussites en mathématiques, physique-chimie, technologie, arts plastiques et EPS, qui témoignent de solides acquis et d'un investissement sérieux. En revanche, sa passivité en classe pénalise ses progrès notamment en anglais et en SVT ; un engagement oral et un travail personnel plus soutenu sont nécessaires. Je l'encourage à conserver son sérieux dans ses bonnes matières tout en étant plus présent et concentré dans les cours moins maîtrisés ; je compte sur lui.

🚨 Alertes (2):
  - [URGENT] SVT: 

In [9]:
# Comparaison côte à côte
print("=" * 60)
print("COMPARAISON SYNTHÈSE")
print("=" * 60)
print(f"\n📝 GROUND TRUTH ({eleve_cible.eleve_id}):")
print(eleve_cible.synthese_ground_truth)
print("\n🤖 GÉNÉRÉE:")
print(synthese.synthese_texte)

COMPARAISON SYNTHÈSE

📝 GROUND TRUTH (ELEVE_C):
Bilan globalement positif mais perfectible. ELEVE_C a fourni un travail sérieux dans de nombreuses matières. Nous comptons désormais sur lui, pour améliorer sa participation en classe et intensifier son travail personnel en SVT et en Anglais afin de réaliser une belle année de 5ème. Nous l'en savons capable.

🤖 GÉNÉRÉE:
ELEVE_C montre de belles réussites en mathématiques, physique-chimie, technologie, arts plastiques et EPS, qui témoignent de solides acquis et d'un investissement sérieux. En revanche, sa passivité en classe pénalise ses progrès notamment en anglais et en SVT ; un engagement oral et un travail personnel plus soutenu sont nécessaires. Je l'encourage à conserver son sérieux dans ses bonnes matières tout en étant plus présent et concentré dans les cours moins maîtrisés ; je compte sur lui.


## 4. Test batch (tous les élèves)

In [ ]:
# Générer pour tous les élèves (leave-one-out: chaque élève est généré sans lui-même en exemple)
resultats = []

for i, eleve in enumerate(eleves):
    # Exemples = tous sauf l'élève courant
    exemples = [e for j, e in enumerate(eleves) if j != i]
    generator.set_exemples(exemples)

    synthese = generator.generate(eleve, classe_info="5ème - Trimestre 1")

    resultats.append(
        {
            "eleve_id": eleve.eleve_id,
            "ground_truth": eleve.synthese_ground_truth,
            "synthese": synthese,  # Objet SyntheseGeneree complet
        }
    )

    print(f"✓ {eleve.eleve_id} généré")

In [ ]:
# Afficher tous les résultats avec insights
for r in resultats:
    s = r["synthese"]
    print("=" * 60)
    print(f"ÉLÈVE: {r['eleve_id']}")
    print("=" * 60)
    print(f"\n📝 GROUND TRUTH:\n{r['ground_truth']}")
    print(f"\n🤖 GÉNÉRÉE:\n{s.synthese_texte}")
    print(f"\n🚨 Alertes: {[f'{a.matiere} ({a.severite})' for a in s.alertes]}")
    print(f"✅ Réussites: {[r.matiere for r in s.reussites]}")
    print(f"🎭 Posture: {s.posture_generale}")
    print(f"🎯 Axes: {s.axes_travail}")
    print()

## 5. Métriques

In [ ]:
# Statistiques basiques
print("Longueur des synthèses:")
for r in resultats:
    gt_len = len(r["ground_truth"])
    gen_len = len(r["synthese"].synthese_texte)
    print(
        f"  {r['eleve_id']}: GT={gt_len} chars, Gen={gen_len} chars, Ratio={gen_len / gt_len:.2f}"
    )

print("\nDistribution des postures:")
from collections import Counter

postures = Counter(r["synthese"].posture_generale for r in resultats)
for posture, count in postures.items():
    print(f"  {posture}: {count}")

print("\nNombre moyen d'alertes/réussites:")
avg_alertes = sum(len(r["synthese"].alertes) for r in resultats) / len(resultats)
avg_reussites = sum(len(r["synthese"].reussites) for r in resultats) / len(resultats)
print(f"  Alertes: {avg_alertes:.1f}")
print(f"  Réussites: {avg_reussites:.1f}")